# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = os.path.join("..", "WeatherPy_database.csv")

# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,73.80,73,79,15.57,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,63.28,90,95,5.53,overcast clouds
2,2,Yatou,CN,37.1500,122.3833,43.32,61,18,6.82,few clouds
3,3,Kapuskasing,CA,49.4169,-82.4331,21.20,86,100,11.50,snow
4,4,Juneau,US,58.3019,-134.4197,36.10,97,100,3.44,light intensity drizzle


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50


In [4]:
# Prompt the user to enter the maximum temperature criteria.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 100


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,73.80,73,79,15.57,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,63.28,90,95,5.53,overcast clouds
6,6,Cabo San Lucas,MX,22.8909,-109.9124,67.51,69,9,3.33,clear sky
8,8,Bredasdorp,ZA,-34.5322,20.0403,71.58,78,77,13.78,broken clouds
9,9,Palmerston,AU,-12.4860,130.9833,85.89,79,75,10.36,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,73.80,73,79,15.57,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,63.28,90,95,5.53,overcast clouds
6,6,Cabo San Lucas,MX,22.8909,-109.9124,67.51,69,9,3.33,clear sky
8,8,Bredasdorp,ZA,-34.5322,20.0403,71.58,78,77,13.78,broken clouds
9,9,Palmerston,AU,-12.4860,130.9833,85.89,79,75,10.36,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Hermanus,ZA,73.80,broken clouds,-34.4187,19.2345
1,Castro,BR,63.28,overcast clouds,-24.7911,-50.0119
6,Cabo San Lucas,MX,67.51,clear sky,22.8909,-109.9124
8,Bredasdorp,ZA,71.58,broken clouds,-34.5322,20.0403
9,Palmerston,AU,85.89,broken clouds,-12.4860,130.9833


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [8]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\aghas\AppData\Local\Temp\ipykernel_9024\1607977541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,73.80,broken clouds,-34.4187,19.2345,
1,Castro,BR,63.28,overcast clouds,-24.7911,-50.0119,
6,Cabo San Lucas,MX,67.51,clear sky,22.8909,-109.9124,
8,Bredasdorp,ZA,71.58,broken clouds,-34.5322,20.0403,
9,Palmerston,AU,85.89,broken clouds,-12.4860,130.9833,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [9]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [14]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["location"] = f"{latitude},{longitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data by using the params dictionaty
    hotels = requests.get(base_url, params=params).json()
     
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [19]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,73.80,broken clouds,-34.4187,19.2345,
1,Castro,BR,63.28,overcast clouds,-24.7911,-50.0119,
6,Cabo San Lucas,MX,67.51,clear sky,22.8909,-109.9124,
8,Bredasdorp,ZA,71.58,broken clouds,-34.5322,20.0403,
9,Palmerston,AU,85.89,broken clouds,-12.4860,130.9833,
12,Celestun,MX,75.56,scattered clouds,20.8667,-90.4000,
13,Ilam,NP,59.07,broken clouds,26.9167,87.9167,
18,Port Alfred,ZA,69.19,light rain,-33.5906,26.8910,
19,Port Elizabeth,ZA,71.89,clear sky,-33.9180,25.5701,
24,Malindi,KE,80.76,scattered clouds,-3.2175,40.1191,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [20]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [21]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [22]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [23]:
clean_hotel_df = pd.read_csv("WeatherPy_vacation.csv")

# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# clean_hotel_df.head()

In [24]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

[]

In [25]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng


In [26]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))